In [ ]:
#!/usr/bin/env python3
import os
import sys
import platform
import argparse
import numpy as np

# --- Use a browser-backed Matplotlib when running in WSL without DISPLAY ---
if platform.system() == "Linux" and not os.environ.get("DISPLAY"):
    import matplotlib as mpl
    mpl.use("WebAgg")
    mpl.rcParams["webagg.open_in_browser"] = True

import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pyproj import Geod

# ---------- Helpers ----------
def is_wsl() -> bool:
    try:
        return "microsoft" in platform.uname().release.lower()
    except Exception:
        return False

def win_to_wsl(path: str) -> str:
    """Convert C:\\foo\\bar -> /mnt/c/foo/bar for WSL."""
    if not path:
        return path
    drive, rest = os.path.splitdrive(path)
    if drive and drive[0].isalpha():
        return f"/mnt/{drive[0].lower()}/{rest.replace('\\', '/')}".replace("//", "/")
    return path.replace("\\", "/")

def ensure_dir_for_file(path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)

def load_to_epsg4326(path):
    """Load GeoTIFF and reproject to EPSG:4326 for display. Returns (img, extent)."""
    with rio.open(path) as src:
        if src.crs is None:
            raise ValueError("Image has no CRS. Provide a georeferenced GeoTIFF.")
        src_crs = src.crs
        dst_crs = "EPSG:4326"

        transform, width, height = calculate_default_transform(
            src_crs, dst_crs, src.width, src.height, *src.bounds
        )
        nb = min(3, src.count) if src.count > 1 else 1
        dst = np.empty((nb, height, width), dtype=src.dtypes[0])

        for i in range(nb):
            reproject(
                source=src.read(i + 1),
                destination=dst[i if nb > 1 else ...],
                src_transform=src.transform,
                src_crs=src_crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest,
            )

    # extent in lon/lat
    left, top = transform.c, transform.f
    right = left + transform.a * dst.shape[-1]
    bottom = top + transform.e * dst.shape[-2]
    extent = [left, right, bottom, top]

    # format image
    if nb >= 3:
        rgb = np.transpose(dst[:3], (1, 2, 0)).astype(np.float32)
        vmax = np.nanmax(rgb)
        if vmax > 0:
            rgb /= vmax
        img = rgb
    else:
        img = dst.astype(np.float32)
        vmax = np.nanmax(img)
        if vmax > 0:
            img /= vmax
    return img, extent

def save_txt_latlon(points_lonlat, out_txt_path):
    """Write lines 'Lat, Lon' (comma + space)."""
    arr = np.asarray(points_lonlat, dtype=float)
    if arr.size:
        latlon = np.column_stack([arr[:, 1], arr[:, 0]])  # (lat, lon)
    else:
        latlon = arr.reshape(0, 2)
    ensure_dir_for_file(out_txt_path)
    with open(out_txt_path, "w", encoding="utf-8") as f:
        for lat, lon in latlon:
            f.write(f"{lat:.7f}, {lon:.7f}\n")
    print(f"Saved {latlon.shape[0]} points to:\n  {out_txt_path}")

def geodesic_length_m(lonlat_pts):
    """Geodesic length (m) over lon/lat points (list of (lon,lat))."""
    if len(lonlat_pts) < 2:
        return 0.0
    g = Geod(ellps="WGS84")
    L = 0.0
    for (lon1, lat1), (lon2, lat2) in zip(lonlat_pts[:-1], lonlat_pts[1:]):
        _, _, dist = g.inv(lon1, lat1, lon2, lat2)
        L += dist
    return L

def hhmm_from_seconds(secs: float) -> str:
    if secs <= 0 or not np.isfinite(secs):
        return "0h 00m"
    h = int(secs // 3600)
    m = int(round((secs - h * 3600) / 60))
    if m == 60:
        h += 1
        m = 0
    return f"{h}h {m:02d}m"

def save_map_png(img, extent, lonlat_pts, out_png_path, speed_mps=1.0):
    """Make a static PNG figure showing raster + picked path in black with length/time overlay."""
    ensure_dir_for_file(out_png_path)
    fig, ax = plt.subplots(figsize=(9, 7))
    if img.ndim == 3:
        ax.imshow(img, extent=extent, origin="upper")
    else:
        ax.imshow(img, extent=extent, origin="upper", cmap="gray")

    ax.set_xlabel("Longitude (°)")
    ax.set_ylabel("Latitude (°)")

    if lonlat_pts:
        xs = [p[0] for p in lonlat_pts]
        ys = [p[1] for p in lonlat_pts]
        ax.plot(xs, ys, "-", linewidth=1.8, color="black")
        ax.scatter(xs, ys, s=20, color="black", zorder=3)

        Lm = geodesic_length_m(lonlat_pts)
        km = Lm / 1000.0
        tsec = Lm / max(speed_mps, 1e-9)
        ax.text(
            0.01, 0.99,
            f"Length: {km:.2f} km\n@{speed_mps:.2f} m/s → {hhmm_from_seconds(tsec)}",
            transform=ax.transAxes, va="top", ha="left",
            bbox=dict(facecolor="white", alpha=0.8, edgecolor="none", boxstyle="round,pad=0.3"),
            fontsize=10
        )

    ax.grid(True, alpha=0.3)
    fig.tight_layout()
    fig.savefig(out_png_path, dpi=200)
    plt.close(fig)
    print(f"Saved map PNG:\n  {out_png_path}")

# ---------- Main ----------
def main():
    ap = argparse.ArgumentParser(add_help=False)
    ap.add_argument("--speed", type=float, default=1.0,
                    help="Speed in m/s for time estimate (default: 1.0)")
    args, _ = ap.parse_known_args()
    speed_mps = float(args.speed) if args.speed and args.speed > 0 else 1.0

    # Ask user for glacier name
    name = input("Enter glacier name (e.g., luggye): ").strip()
    if not name:
        print("No name provided. Exiting.")
        sys.exit(1)

    # Windows canonical paths
    win_image = fr"C:\duckling\satellite_images\{name}.tif"
    win_txt   = fr"C:\duckling\waypoints\{name}_waypoints.txt"
    win_png   = fr"C:\duckling\waypoints\{name}_waypoints.png"

    # Convert to WSL paths if applicable
    if platform.system() == "Linux" and (is_wsl() or os.path.exists("/mnt")):
        image_path = win_to_wsl(win_image)
        txt_path   = win_to_wsl(win_txt)
        png_path   = win_to_wsl(win_png)
    else:
        image_path = win_image
        txt_path   = win_txt
        png_path   = win_png

    print("Resolved paths:")
    print(f"  Image:  {image_path}")
    print(f"  Output: {txt_path}")
    print(f"  Figure: {png_path}")
    print(f"  Speed:  {speed_mps:.2f} m/s (for time estimate)")

    if not os.path.exists(image_path):
        print(f"\nERROR: Image not found:\n  {image_path}")
        sys.exit(1)

    # Load raster (lon/lat) once
    img, extent = load_to_epsg4326(image_path)

    # Interactive picking (WebAgg opens in browser if in WSL)
    fig, ax = plt.subplots(figsize=(9, 7))
    if img.ndim == 3:
        ax.imshow(img, extent=extent, origin="upper")
    else:
        ax.imshow(img, extent=extent, origin="upper", cmap="gray")
    ax.set_xlabel("Longitude (°)")
    ax.set_ylabel("Latitude (°)")
    ax.set_title("Left-click: add point | u: undo | r: reset | f: save & finish")

    points = []  # (lon, lat)
    scat = None
    line = None
    overlay = ax.text(
        0.01, 0.99, "Length: 0.00 km\n@{:.2f} m/s → 0h 00m".format(speed_mps),
        transform=ax.transAxes, va="top", ha="left",
        bbox=dict(facecolor="white", alpha=0.8, edgecolor="none", boxstyle="round,pad=0.3"),
        fontsize=10
    )

    def update_overlay():
        Lm = geodesic_length_m(points)
        km = Lm / 1000.0
        tsec = Lm / max(speed_mps, 1e-9)
        overlay.set_text(f"Length: {km:.2f} km\n@{speed_mps:.2f} m/s → {hhmm_from_seconds(tsec)}")

    def redraw():
        nonlocal scat, line
        if scat is not None:
            scat.remove(); scat = None
        if line is not None:
            line.remove(); line = None
        if points:
            xs, ys = zip(*points)
            scat = ax.scatter(xs, ys, s=30, marker="o", color="black")
            if len(points) >= 2:
                line = ax.plot(xs, ys, "-", linewidth=1.6, color="black")[0]
        update_overlay()
        fig.canvas.draw_idle()

    def on_click(event):
        if event.inaxes != ax or event.button != 1:
            return
        if event.xdata is None or event.ydata is None:
            return
        points.append((float(event.xdata), float(event.ydata)))
        redraw()

    def on_key(event):
        if event.key == "u":
            if points:
                points.pop(); redraw()
        elif event.key == "r":
            points.clear(); redraw()
        elif event.key == "f":
            # Save TXT
            save_txt_latlon(points, txt_path)
            # Save static PNG map (black track + overlay)
            save_map_png(img, extent, points, png_path, speed_mps=speed_mps)
            # Print summary in terminal
            Lm = geodesic_length_m(points)
            print(f"Length: {Lm/1000.0:.2f} km • @{speed_mps:.2f} m/s → {hhmm_from_seconds(Lm/max(speed_mps,1e-9))}")
            plt.close(fig)

    fig.canvas.mpl_connect("button_press_event", on_click)
    fig.canvas.mpl_connect("key_press_event", on_key)

    print("\nControls:")
    print("  Left-click: add point")
    print("  u: undo last point")
    print("  r: reset all points")
    print("  f: save & finish")
    print("\nA browser window should open automatically if running under WSL (WebAgg).")

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()


In [ ]:
# !jupyter nbconvert --to script click_path.ipynb